<a href="https://colab.research.google.com/github/annkvth/kompleteNetze/blob/main/TrafficTrinoTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

See example from the documentation: https://traffic-viz.github.io/gallery/kaliningrad.html

Note: there's no need to store username and password in a config file - the jupyter output will feature a link for trino authentication in a separate browser window

Installing the traffic python package if necessary


In [ ]:
!pip install traffic



Include libraries

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
#from cartes.crs import Lambert93  # type: ignore

Always use python engine for dataframe queries, see
https://stackoverflow.com/questions/67063643/is-there-a-way-to-force-pandas-dataframe-query-to-use-python-as-default-engine

In [ ]:
from functools import partialmethod

pd.DataFrame.query = partialmethod(pd.DataFrame.query, engine="python")



... and the traffic library used to hadle the opensky queries

In [ ]:
import traffic
traffic.config_file

In [ ]:
from traffic.data import opensky
#from traffic.core import Flight
#from traffic.data import eurofirs


In [ ]:
t = opensky.history(
    "2022-02-22",
    "2022-03-06",
    callsign=[
        # Some callsigns between Moscow, Saint Petersburg and Kaliningrad
        "AFL1000 ", "AFL1002 ", "AFL1004 ", "AFL1006 ", "AFL1008 ",
        "SDM6323 ", "SDM6325 ", "SDM6329 ",
    ],
)


In [ ]:
import matplotlib.pyplot as plt

from cartes.crs import LambertConformal
from cartes.utils.features import countries, lakes, ocean

from traffic.data import eurofirs, airports

fig, ax = plt.subplots(
    1, 2, figsize=(15, 6), dpi=300,
    subplot_kw=dict(projection=LambertConformal(10, 45)),
)

title_style = dict(loc="left", font="Ubuntu", fontsize=16, fontweight=400)

for ax_ in ax:
    ax_.set_extent((15, 40, 54, 60))

    ax_.add_feature(countries(scale="50m"))
    ax_.add_feature(lakes(scale="50m"))
    ax_.add_feature(ocean(scale="50m"))

    ax_.spines["geo"].set_visible(False)

    for fir in eurofirs:
        fir.plot(ax_, color="#4c78a833")
        fir.plot(ax_, edgecolor="#4c78a8")

    airports["UUEE"].point.plot(ax_, text_kw=dict(s="  Moscow", fontsize=12))
    airports["ULLI"].point.plot(ax_, text_kw=dict(s="  Saint Petersburg", fontsize=12))
    airports["UMKK"].point.plot(ax_, text_kw=dict(s="  Kaliningrad", fontsize=12))


t.before("2022-02-27").plot(ax[0], color="#f58518")
ax[0].set_title("Before February 27", **title_style)
t.after("2022-02-28").plot(ax[1], color="#f58518")
ax[1].set_title("After February 28", **title_style)

fig.set_tight_layout(True)